In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/03 20:29:42 WARN Utils: Your hostname, neo4j-testing resolves to a loopback address: 127.0.1.1; using 10.0.0.24 instead (on interface enp0s3)
23/03/03 20:29:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/03 20:29:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.3.2'

In [4]:
!ls -lh data/

total 168M
-rw-rw-r-- 1 ubuntu ubuntu 168M Dec 20 00:13 fhvhv_tripdata_2021-06.csv.gz
-rw-rw-r-- 1 ubuntu ubuntu  13K Jul 19  2022 taxi_zone_lookup.csv


In [7]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
    
])

In [8]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('data/fhvhv_tripdata_2021-06.csv.gz')

df = df.repartition(12)

df.write.parquet('data/pq/fhvhv/2021/06/', compression="gzip")

In [9]:
! ls -lh data/pq/fhvhv/2021/06/

total 240M
-rw-r--r-- 1 ubuntu ubuntu   0 Mar  3 20:31 _SUCCESS
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:30 part-00000-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:30 part-00001-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:30 part-00002-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:30 part-00003-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:31 part-00004-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:31 part-00005-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:31 part-00006-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:31 part-00007-11c847ed-4e41-42f7-a0f0-1f622a483bd0-c000.gz.parquet
-rw-r--r-- 1 ubuntu ubuntu 20M Mar  3 20:31 part-00008-11c847ed-4e41-42f

In [10]:
from pyspark.sql import functions as F

In [11]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .filter("pickup_date = '2021-06-15'") \
    .count()

452470

In [12]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [14]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(1) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-25|       240764|
+-----------+-------------+



In [15]:
df.registerTempTable('fhvhv_2021_06')

/home/ubuntu/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [18]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 3600) AS duration
FROM 
    fhvhv_2021_06
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 1;
""").show()

+-----------+----------------+
|pickup_date|        duration|
+-----------+----------------+
| 2021-06-25|66.8788888888889|
+-----------+----------------+



In [24]:
schema_zones = types.StructType([
    types.StructField('LocationID', types.StringType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True),
    
])

In [25]:
df_zones = spark.read \
            .option("header", "true") \
            .schema(schema_zones) \
            .csv("data/taxi_zone_lookup.csv")

In [26]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [27]:
df_zones.registerTempTable('zones')

/home/ubuntu/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [29]:
spark.sql("""
SELECT
    pul.Zone,
    COUNT(1)
FROM 
    fhvhv_2021_06 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
GROUP BY 
    1
ORDER BY
    2 DESC
LIMIT 5;
""").show()

+-------------------+--------+
|               Zone|count(1)|
+-------------------+--------+
|Crown Heights North|  231279|
|       East Village|  221244|
|        JFK Airport|  188867|
|     Bushwick South|  187929|
|      East New York|  186780|
+-------------------+--------+

